In [1]:
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1"# [NEW] Extra 30% context lengths!

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8000 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "LiquidAI/LFM2-350M",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = False, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.9, # Reduce if out of memory

    use_exact_model_name = True #for hugginface cache or repo mdoel name
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

/home/ng6309/datascience/anand/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 10-08 00:15:45 [__init__.py:216] Automatically detected platform cuda.


W1008 00:15:45.401000 369457 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W1008 00:15:45.401000 369457 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


Switching to PyTorch attention since your Xformers is broken.

Requires Flash-Attention version >=2.7.1,<=2.8.2 but got 2.8.3.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.9: Fast Lfm2 patching. Transformers: 4.56.2. vLLM: 0.10.2.
   \\   /|    NVIDIA RTX 6000 Ada Generation. Num GPUs = 1. Max memory: 47.507 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.
Unsloth: Making `model.base_model.model.model` require gradients


In [3]:
tokenizer.chat_template

'{{- bos_token -}}\n{%- set system_prompt = "" -%}\n{%- set ns = namespace(system_prompt="") -%}\n{%- if messages[0]["role"] == "system" -%}\n\t{%- set ns.system_prompt = messages[0]["content"] -%}\n\t{%- set messages = messages[1:] -%}\n{%- endif -%}\n{%- if tools -%}\n\t{%- set ns.system_prompt = ns.system_prompt + ("\\n" if ns.system_prompt else "") + "List of tools: <|tool_list_start|>[" -%}\n\t{%- for tool in tools -%}\n\t\t{%- if tool is not string -%}\n            {%- set tool = tool | tojson -%}\n\t\t{%- endif -%}\n\t\t{%- set ns.system_prompt = ns.system_prompt + tool -%}\n        {%- if not loop.last -%}\n            {%- set ns.system_prompt = ns.system_prompt + ", " -%}\n        {%- endif -%}\n\t{%- endfor -%}\n\t{%- set ns.system_prompt = ns.system_prompt + "]<|tool_list_end|>" -%}\n{%- endif -%}\n{%- if ns.system_prompt -%}\n\t{{- "<|im_start|>system\\n" + ns.system_prompt + "<|im_end|>\\n" -}}\n{%- endif -%}\n{%- for message in messages -%}\n\t{{- "<|im_start|>" + message

In [18]:
# Clean GLM-style template with line-by-line tool format
glm_chat_template = '''{{- bos_token -}}
{%- set system_prompt = "" -%}
{%- set ns = namespace(system_prompt="") -%}
{%- if messages[0]["role"] == "system" -%}
	{%- set ns.system_prompt = messages[0]["content"] -%}
	{%- set messages = messages[1:] -%}
{%- endif -%}
{%- if tools -%}
	{%- set ns.system_prompt = ns.system_prompt + ("\\n" if ns.system_prompt else "") + "# Tools\\nYou may call one or more functions to assist with the user query.\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>\\n" -%}
	{%- for tool in tools -%}
		{%- if tool is not string -%}
			{%- set tool = tool.function | tojson -%}
		{%- endif -%}
		{%- set ns.system_prompt = ns.system_prompt + tool -%}
		{%- if not loop.last -%}
			{%- set ns.system_prompt = ns.system_prompt + "\\n" -%}
		{%- endif -%}
	{%- endfor -%}
	{%- set ns.system_prompt = ns.system_prompt + "\\n</tools>\\nFor each function call, output the function name and arguments within the following XML format:\\n<tool_call>{function-name}\\n<arg_key>{arg_key}</arg_key>\\n<arg_value>{arg_value}</arg_value>\\n...\\n</tool_call>" -%}
{%- endif -%}
{%- if ns.system_prompt -%}
	{{- "<|im_start|>system\\n" + ns.system_prompt + "<|im_end|>\\n" -}}
{%- endif -%}
{%- for message in messages -%}
	{{- "<|im_start|>" + message["role"] + "\\n" -}}
	{%- set content = message["content"] -%}
	{%- if content is not string -%}
		{%- set content = content | tojson -%}
	{%- endif -%}
	{%- if message["role"] == "tool" -%}
		{%- set content = "<|tool_response_start|>" + content + "<|tool_response_end|>" -%}
	{%- endif -%}
	{{- content + "<|im_end|>\\n" -}}
{%- endfor -%}
{%- if add_generation_prompt -%}
	{{- "<|im_start|>assistant\\n" -}}
{%- endif -%}'''

tokenizer.chat_template = glm_chat_template

In [21]:
# Complete conversation with user, assistant, and tool messages (no think tags)
messages = [
    {"role": "user", "content": "What is the current status of candidate ID 12345?"},
    {"role": "assistant", "content": '<tool_call>get_candidate_status\n<arg_key>candidate_id</arg_key>\n<arg_value>12345</arg_value>\n</tool_call>'},
    {"role": "tool", "content": '{"candidate_id": "12345", "status": "Interview Scheduled", "position": "Clinical Research Associate", "date": "2023-11-20"}'},
    {"role": "assistant", "content": "The candidate with ID 12345 is currently in the \"Interview Scheduled\" stage for the position of Clinical Research Associate, with an interview date set for 2023-11-20."},
    {"role": "user", "content": "Can you also search for all candidates for the position of Data Scientist?"},
    {"role": "assistant", "content": '<tool_call>search_database\n<arg_key>query</arg_key>\n<arg_value>Data Scientist</arg_value>\n<arg_key>limit</arg_key>\n<arg_value>10</arg_value>\n</tool_call>'},
    {"role": "tool", "content": '[{"candidate_id": "67890", "name": "John Doe", "status": "Applied"}, {"candidate_id": "67891", "name": "Jane Smith", "status": "Interview Completed"}]'},
    {"role": "assistant", "content": "I found 2 candidates for the Data Scientist position:\n1. John Doe (ID: 67890) - Status: Applied\n2. Jane Smith (ID: 67891) - Status: Interview Completed"}
]

formatted = tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=False
)

print(formatted)

<|startoftext|><|im_start|>user
What is the current status of candidate ID 12345?<|im_end|>
<|im_start|>assistant
<tool_call>get_candidate_status
<arg_key>candidate_id</arg_key>
<arg_value>12345</arg_value>
</tool_call><|im_end|>
<|im_start|>tool
<|tool_response_start|>{"candidate_id": "12345", "status": "Interview Scheduled", "position": "Clinical Research Associate", "date": "2023-11-20"}<|tool_response_end|><|im_end|>
<|im_start|>assistant
The candidate with ID 12345 is currently in the "Interview Scheduled" stage for the position of Clinical Research Associate, with an interview date set for 2023-11-20.<|im_end|>
<|im_start|>user
Can you also search for all candidates for the position of Data Scientist?<|im_end|>
<|im_start|>assistant
<tool_call>search_database
<arg_key>query</arg_key>
<arg_value>Data Scientist</arg_value>
<arg_key>limit</arg_key>
<arg_value>10</arg_value>
</tool_call><|im_end|>
<|im_start|>tool
<|tool_response_start|>[{"candidate_id": "67890", "name": "John Doe", 

In [16]:
base_tools = {
  "web_search": {
    "type": "function",
    "function": {
      "name": "web_search",
      "description": "Search the web",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "Search query"
          }
        },
        "required": [
          "query"
        ]
      },
      "return": {
        "type": "array",
        "items": {
          "type": "object"
        },
        "description": "The list of items having url,title,description."
      }
    }
  },
  "think": {
    "type": "function",
    "function": {
      "name": "think",
      "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.",
      "parameters": {
        "type": "object",
        "properties": {
          "thought": {
            "type": "string",
            "description": "A thought to think about."
          }
        },
        "required": [
          "thought"
        ]
      }
    }
  },
  "response": {
    "type": "function",
    "function": {
      "name": "response",
      "description": "Send a message back to the user.",
      "parameters": {
        "type": "object",
        "properties": {
          "message": {
            "type": "string",
            "description": "The message to send to the user."
          }
        },
        "required": [
          "message"
        ]
      }
    }
  }
}

In [22]:
# Complete conversation with user, assistant, and tool messages (no think tags)
messages = [
    {"role": "user", "content": "hiiii"}
]

formatted = tokenizer.apply_chat_template(
    messages, 
    tools=list(base_tools.values()), 
    tokenize=False, 
    add_generation_prompt=True
)

print(formatted)

<|startoftext|><|im_start|>system
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"name": "web_search", "description": "Search the web", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}, "return": {"type": "array", "items": {"type": "object"}, "description": "The list of items having url,title,description."}}
{"name": "think", "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.", "parameters": {"type": "object", "properties": {"thought": {"type": "string", "description": "A thought to think about."}}, "required": ["thought"]}}
{"name": "response", "description": "Send a message back to the user.", "parameters": {"type": "object", "prop

## Chattemplate Explored

In [4]:
def create_glm_tool_template():
    # GLM-4.6 style components
    components = {
        'think_start': '<think>',
        'think_end': '</think>',
        'tool_start': '<tool_call>',
        'tool_end': '</tool_call>',
        'arg_key_start': '<arg_key>',
        'arg_key_end': '</arg_key>',
        'arg_value_start': '<arg_value>',
        'arg_value_end': '</arg_value>',
        'observation_start': '<|observation|>',
        'system_start': '<|system|>',
        'user_start': '<|user|>',
        'assistant_start': '<|assistant|>',
        'solution_start': '<SOLUTION>',
        'solution_end': '</SOLUTION>'
    }
    
    system_prompt_with_tools = f"""{components['system_start']}
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{{"name": "get_weather", "description": "Get the weather of a city for a specific date.", "parameters": {{"type": "object", "properties": {{"city": {{"type": "string", "description": "The city to get weather for, in Chinese."}}, "date": {{"type": "string", "description": "The date in YYYY-MM-DD format."}}}}, "required": ["city"]}}}}
</tools>
For each function call, output the function name and arguments within the following XML format:
{components['tool_start']}{{function-name}}
{components['arg_key_start']}{{arg_key}}{components['arg_key_end']}
{components['arg_value_start']}{{arg_value}}{components['arg_value_end']}
...
{components['tool_end']}{components['system_start']}
You are a helpful assistant."""
    
    chat_template = f"""{{% if messages[0]['role'] == 'system' %}}
{{ messages[0]['content'] + eos_token }}
{{% set loop_messages = messages[1:] %}}
{{% else %}}
{{ '{system_prompt_with_tools}' + eos_token }}
{{% set loop_messages = messages %}}
{{% endif %}}
{{% for message in loop_messages %}}
{{% if message['role'] == 'user' %}}
{{ '<|user|>' + message['content'] }}
{{% elif message['role'] == 'assistant' %}}
{{ '<|assistant|>' + message['content'] + eos_token }}
{{% elif message['role'] == 'observation' %}}
{{ '<|observation|>' + message['content'] + '<|assistant|>' }}
{{% endif %}}
{{% endfor %}}
{{% if add_generation_prompt %}}{{ '<|assistant|>' }}{{% endif %}}"""
    
    return chat_template

# Apply the template
tokenizer.chat_template = create_glm_tool_template()

# Example usage:
messages = [
    {"role": "user", "content": "Today is June 26, 2024. Could you please check the weather in Beijing and Shanghai for tomorrow?"},
    {"role": "assistant", "content": "<think>The user wants to check the weather of Beijing and Shanghai tomorrow. I need to call the get_weather function respectively to check Beijing and Shanghai.</think>\nI will call the get_weather function to check the weather in Beijing and Shanghai.\n<tool_call>get_weather\n<arg_key>city</arg_key>\n<arg_value>Beijing</arg_value>\n<arg_key>date</arg_key>\n<arg_value>2024-06-27</arg_value>\n</tool_call>\n<tool_call>get_weather\n<arg_key>city</arg_key>\n<arg_value>Shanghai</arg_value>\n<arg_key>date</arg_key>\n<arg_value>2024-06-27</arg_value>\n</tool_call>"},
    {"role": "observation", "content": '[{"city": "Beijing", "date": "2024-06-27", "weather": "Sunny", "temperature": "26C"}, {"city": "Shanghai", "date": "2024-06-27", "weather": "Overcast", "temperature": "29C"}]'},
]

messages = [
    {"role": "user", "content": "hi"},
]

formatted = tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)
print(formatted)

{ '<|system|>
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"name": "get_weather", "description": "Get the weather of a city for a specific date.", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": "The city to get weather for, in Chinese."}, "date": {"type": "string", "description": "The date in YYYY-MM-DD format."}}, "required": ["city"]}}
</tools>
For each function call, output the function name and arguments within the following XML format:
<tool_call>{function-name}
<arg_key>{arg_key}</arg_key>
<arg_value>{arg_value}</arg_value>
...
</tool_call><|system|>
You are a helpful assistant.' + eos_token }
{ '<|user|>' + message['content'] }
{ '<|assistant|>' }


In [5]:
def create_advanced_glm_compatible_template():
    chat_template = """{% if messages[0]['role'] == 'system' %}
{{ '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>' }}
{% set loop_messages = messages[1:] %}
{% else %}
{{ '<|im_start|>system\nList of tools: <|tool_list_start|>[{"name": "get_candidate_status", "description": "Retrieves the current status of a candidate in the recruitment process", "parameters": {"type": "object", "properties": {"candidate_id": {"type": "string", "description": "Unique identifier for the candidate"}}, "required": ["candidate_id"]}}]<|tool_list_end|><|im_end|>' }}
{% set loop_messages = messages %}
{% endif %}
{% for message in loop_messages %}
{% if message['role'] == 'user' %}
{{ '<|im_start|>user\n' + message['content'] + '<|im_end|>' }}
{% elif message['role'] == 'assistant' %}
{{ '<|im_start|>assistant\n' + message['content'] + '<|im_end|>' }}
{% elif message['role'] == 'tool' or message['role'] == 'observation' %}
{{ '<|im_start|>tool\n<|tool_response_start|>' + message['content'] + '<|tool_response_end|><|im_end|>' }}
{% endif %}
{% endfor %}
{% if add_generation_prompt %}{{ '<|im_start|>assistant\n<think>' }}{% endif %}"""
    
    return chat_template

tokenizer.chat_template = create_advanced_glm_compatible_template()

# Example with GLM-style tool calls:
messages = [
    {"role": "user", "content": "What is the current status of candidate ID 12345?"},
    {"role": "assistant", "content": '<think>The user wants to check the status of candidate ID 12345. I need to use the get_candidate_status tool.</think>\n<tool_call>get_candidate_status\n<arg_key>candidate_id</arg_key>\n<arg_value>12345</arg_value>\n</tool_call>'},
    {"role": "tool", "content": '{"candidate_id": "12345", "status": "Interview Scheduled", "position": "Clinical Research Associate", "date": "2023-11-20"}'},
    {"role": "assistant", "content": "The candidate with ID 12345 is currently in the \"Interview Scheduled\" stage for the position of Clinical Research Associate, with an interview date set for 2023-11-20."}
]

formatted = tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)
print(formatted)

<|im_start|>system
List of tools: <|tool_list_start|>[{"name": "get_candidate_status", "description": "Retrieves the current status of a candidate in the recruitment process", "parameters": {"type": "object", "properties": {"candidate_id": {"type": "string", "description": "Unique identifier for the candidate"}}, "required": ["candidate_id"]}}]<|tool_list_end|><|im_end|>
<|im_start|>user
What is the current status of candidate ID 12345?<|im_end|>
<|im_start|>assistant
<think>The user wants to check the status of candidate ID 12345. I need to use the get_candidate_status tool.</think>
<tool_call>get_candidate_status
<arg_key>candidate_id</arg_key>
<arg_value>12345</arg_value>
</tool_call><|im_end|>
<|im_start|>tool
<|tool_response_start|>{"candidate_id": "12345", "status": "Interview Scheduled", "position": "Clinical Research Associate", "date": "2023-11-20"}<|tool_response_end|><|im_end|>
<|im_start|>assistant
The candidate with ID 12345 is currently in the "Interview Scheduled" stage 

In [6]:
def create_glm_tool_template(tools_list=None):
    import json
    tools_json = json.dumps(tools_list, ensure_ascii=False)
    
    system_prompt_with_tools = f"""<|system|>
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{tools_json}
</tools>
For each function call, output the function name and arguments within the following XML format:
<tool_call>{{function-name}}
<arg_key>{{arg_key}}</arg_key>
<arg_value>{{arg_value}}</arg_value>
...
</tool_call><|system|>
You are a helpful assistant."""
    
    chat_template = (
        "{% if messages[0]['role'] == 'system' %}"
        "{{ messages[0]['content'] + eos_token }}"
        "{% set loop_messages = messages[1:] %}"
        "{% else %}"
        "{{ '" + system_prompt_with_tools.replace('\n', '\\n').replace('"', '\\"') + "' + eos_token }}"
        "{% set loop_messages = messages %}"
        "{% endif %}"
        "{% for message in loop_messages %}"
        "{% if message['role'] == 'user' %}"
        "{{ '<|user|>' + message['content'] }}"
        "{% elif message['role'] == 'assistant' %}"
        "{{ '<|assistant|>' + message['content'] + eos_token }}"
        "{% elif message['role'] == 'observation' %}"
        "{{ '<|observation|>' + message['content'] + '<|assistant|>' }}"
        "{% endif %}"
        "{% endfor %}"
        "{% if add_generation_prompt %}{{ '<|assistant|>' }}{% endif %}"
    )
    
    return chat_template

# Example with custom tools
custom_tools = [
    {
        "name": "get_candidate_status", 
        "description": "Retrieves the current status of a candidate in the recruitment process", 
        "parameters": {
            "type": "object", 
            "properties": {
                "candidate_id": {"type": "string", "description": "Unique identifier for the candidate"}
            }, 
            "required": ["candidate_id"]
        }
    },
    {
        "name": "search_database",
        "description": "Search the database for specific information",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "The search query"},
                "limit": {"type": "integer", "description": "Number of results to return"}
            },
            "required": ["query"]
        }
    }
]

# Create template with custom tools
tokenizer.chat_template = create_glm_tool_template()

# Test
messages = [
    {"role": "user", "content": "hi"},
]

formatted = tokenizer.apply_chat_template(
    messages, 
    tools=custom_tools,
    tokenize=False, 
    add_generation_prompt=True
)
print(formatted)

print("\n" + "="*50 + "\n")

# Test with a full conversation
messages_with_tools = [
    {"role": "user", "content": "What is the current status of candidate ID 12345?"},
    {"role": "assistant", "content": '<think>The user wants to check the status of candidate ID 12345. I need to use the get_candidate_status tool.</think>\n<tool_call>get_candidate_status\n<arg_key>candidate_id</arg_key>\n<arg_value>12345</arg_value>\n</tool_call>'},
    {"role": "observation", "content": '{"candidate_id": "12345", "status": "Interview Scheduled", "position": "Clinical Research Associate", "date": "2023-11-20"}'},
    {"role": "assistant", "content": "The candidate with ID 12345 is currently in the \"Interview Scheduled\" stage for the position of Clinical Research Associate, with an interview date set for 2023-11-20."}
]

formatted_full = tokenizer.apply_chat_template(
    messages_with_tools, 
    tokenize=False, 
    add_generation_prompt=True
)
print(formatted_full)

<|system|>
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
null
</tools>
For each function call, output the function name and arguments within the following XML format:
<tool_call>{function-name}
<arg_key>{arg_key}</arg_key>
<arg_value>{arg_value}</arg_value>
...
</tool_call><|system|>
You are a helpful assistant.<|im_end|><|user|>hi<|assistant|>


<|system|>
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
null
</tools>
For each function call, output the function name and arguments within the following XML format:
<tool_call>{function-name}
<arg_key>{arg_key}</arg_key>
<arg_value>{arg_value}</arg_value>
...
</tool_call><|system|>
You are a helpful assistant.<|im_end|><|user|>What is the current status of candidate ID 12345?<|assistant|><think>The user wants to check the s

In [7]:
# GLM-style template that preserves the tools parameter functionality
glm_chat_template = (
    '{{- bos_token -}}\n'
    '{%- set system_prompt = "" -%}\n'
    '{%- set ns = namespace(system_prompt="") -%}\n'
    '{%- if messages[0]["role"] == "system" -%}\n'
    '\t{%- set ns.system_prompt = messages[0]["content"] -%}\n'
    '\t{%- set messages = messages[1:] -%}\n'
    '{%- endif -%}\n'
    '{%- if tools -%}\n'
    '\t{%- set ns.system_prompt = ns.system_prompt + ("\\n" if ns.system_prompt else "") + "# Tools\\nYou may call one or more functions to assist with the user query.\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>[" -%}\n'
    '\t{%- for tool in tools -%}\n'
    '\t\t{%- if tool is not string -%}\n'
    '\t\t\t{%- set tool = tool | tojson -%}\n'
    '\t\t{%- endif -%}\n'
    '\t\t{%- set ns.system_prompt = ns.system_prompt + tool -%}\n'
    '\t\t{%- if not loop.last -%}\n'
    '\t\t\t{%- set ns.system_prompt = ns.system_prompt + ", " -%}\n'
    '\t\t{%- endif -%}\n'
    '\t{%- endfor -%}\n'
    '\t{%- set ns.system_prompt = ns.system_prompt + "]</tools>\\nFor each function call, output the function name and arguments within the following XML format:\\n<tool_call>{function-name}\\n<arg_key>{arg_key}</arg_key>\\n<arg_value>{arg_value}</arg_value>\\n...\\n</tool_call>" -%}\n'
    '{%- endif -%}\n'
    '{%- if ns.system_prompt -%}\n'
    '\t{{- "<|im_start|>system\\n" + ns.system_prompt + "<|im_end|>\\n" -}}\n'
    '{%- endif -%}\n'
    '{%- for message in messages -%}\n'
    '\t{{- "<|im_start|>" + message["role"] + "\\n" -}}\n'
    '\t{%- set content = message["content"] -%}\n'
    '\t{%- if content is not string -%}\n'
    '\t\t{%- set content = content | tojson -%}\n'
    '\t{%- endif -%}\n'
    '\t{%- if message["role"] == "tool" -%}\n'
    '\t\t{%- set content = "<|tool_response_start|>" + content + "<|tool_response_end|>" -%}\n'
    '\t{%- endif -%}\n'
    '\t{{- content + "<|im_end|>\\n" -}}\n'
    '{%- endfor -%}\n'
    '{%- if add_generation_prompt -%}\n'
    '\t{{- "<|im_start|>assistant\\n<think>" -}}\n'
    '{%- endif -%}'
)

# Apply the new template
tokenizer.chat_template = glm_chat_template

# Test it
messages = [
    {"role": "user", "content": "hi"},
]

custom_tools = [
    {
        "name": "get_candidate_status", 
        "description": "Retrieves the current status of a candidate in the recruitment process", 
        "parameters": {
            "type": "object", 
            "properties": {
                "candidate_id": {"type": "string", "description": "Unique identifier for the candidate"}
            }, 
            "required": ["candidate_id"]
        }
    }
]

# This should now work with tools parameter
formatted = tokenizer.apply_chat_template(
    messages, 
    tools=custom_tools, 
    tokenize=False, 
    add_generation_prompt=True
)
print(formatted)

<|startoftext|><|im_start|>system
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>[{"name": "get_candidate_status", "description": "Retrieves the current status of a candidate in the recruitment process", "parameters": {"type": "object", "properties": {"candidate_id": {"type": "string", "description": "Unique identifier for the candidate"}}, "required": ["candidate_id"]}}]</tools>
For each function call, output the function name and arguments within the following XML format:
<tool_call>{function-name}
<arg_key>{arg_key}</arg_key>
<arg_value>{arg_value}</arg_value>
...
</tool_call><|im_end|>
<|im_start|>user
hi<|im_end|>
<|im_start|>assistant
<think>


In [10]:
# Make sure we have the GLM template applied
glm_chat_template = (
    '{{- bos_token -}}\n'
    '{%- set system_prompt = "" -%}\n'
    '{%- set ns = namespace(system_prompt="") -%}\n'
    '{%- if messages[0]["role"] == "system" -%}\n'
    '\t{%- set ns.system_prompt = messages[0]["content"] -%}\n'
    '\t{%- set messages = messages[1:] -%}\n'
    '{%- endif -%}\n'
    '{%- if tools -%}\n'
    '\t{%- set ns.system_prompt = ns.system_prompt + ("\\n" if ns.system_prompt else "") + "# Tools\\nYou may call one or more functions to assist with the user query.\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>[" -%}\n'
    '\t{%- for tool in tools -%}\n'
    '\t\t{%- if tool is not string -%}\n'
    '\t\t\t{%- set tool = tool | tojson -%}\n'
    '\t\t{%- endif -%}\n'
    '\t\t{%- set ns.system_prompt = ns.system_prompt + tool -%}\n'
    '\t\t{%- if not loop.last -%}\n'
    '\t\t\t{%- set ns.system_prompt = ns.system_prompt + ", " -%}\n'
    '\t\t{%- endif -%}\n'
    '\t{%- endfor -%}\n'
    '\t{%- set ns.system_prompt = ns.system_prompt + "]</tools>\\nFor each function call, output the function name and arguments within the following XML format:\\n<tool_call>{function-name}\\n<arg_key>{arg_key}</arg_key>\\n<arg_value>{arg_value}</arg_value>\\n...\\n</tool_call>" -%}\n'
    '{%- endif -%}\n'
    '{%- if ns.system_prompt -%}\n'
    '\t{{- "<|im_start|>system\\n" + ns.system_prompt + "<|im_end|>\\n" -}}\n'
    '{%- endif -%}\n'
    '{%- for message in messages -%}\n'
    '\t{{- "<|im_start|>" + message["role"] + "\\n" -}}\n'
    '\t{%- set content = message["content"] -%}\n'
    '\t{%- if content is not string -%}\n'
    '\t\t{%- set content = content | tojson -%}\n'
    '\t{%- endif -%}\n'
    '\t{%- if message["role"] == "tool" -%}\n'
    '\t\t{%- set content = "<|tool_response_start|>" + content + "<|tool_response_end|>" -%}\n'
    '\t{%- endif -%}\n'
    '\t{{- content + "<|im_end|>\\n" -}}\n'
    '{%- endfor -%}\n'
    '{%- if add_generation_prompt -%}\n'
    '\t{{- "<|im_start|>assistant\\n<think>" -}}\n'
    '{%- endif -%}'
)

tokenizer.chat_template = glm_chat_template

# Example with complete conversation flow
custom_tools = [
    {
        "name": "get_candidate_status", 
        "description": "Retrieves the current status of a candidate in the recruitment process", 
        "parameters": {
            "type": "object", 
            "properties": {
                "candidate_id": {"type": "string", "description": "Unique identifier for the candidate"}
            }, 
            "required": ["candidate_id"]
        }
    },
    {
        "name": "search_database",
        "description": "Search the database for specific information",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "The search query"},
                "limit": {"type": "integer", "description": "Number of results to return"}
            },
            "required": ["query"]
        }
    }
]

# Complete conversation with user, assistant, and tool messages
messages = [
    {"role": "user", "content": "What is the current status of candidate ID 12345?"},
    {"role": "assistant", "content": '<think>The user wants to check the status of candidate ID 12345. I need to use the get_candidate_status tool.</think>\n<tool_call>get_candidate_status\n<arg_key>candidate_id</arg_key>\n<arg_value>12345</arg_value>\n</tool_call>'},
    {"role": "tool", "content": '{"candidate_id": "12345", "status": "Interview Scheduled", "position": "Clinical Research Associate", "date": "2023-11-20"}'},
    {"role": "assistant", "content": "The candidate with ID 12345 is currently in the \"Interview Scheduled\" stage for the position of Clinical Research Associate, with an interview date set for 2023-11-20."},
    {"role": "user", "content": "Can you also search for all candidates for the position of Data Scientist?"},
    {"role": "assistant", "content": '<think>The user wants to search for all candidates for the Data Scientist position. I need to use the search_database tool.</think>\n<tool_call>search_database\n<arg_key>query</arg_key>\n<arg_value>Data Scientist</arg_value>\n<arg_key>limit</arg_key>\n<arg_value>10</arg_value>\n</tool_call>'},
    {"role": "tool", "content": '[{"candidate_id": "67890", "name": "John Doe", "status": "Applied"}, {"candidate_id": "67891", "name": "Jane Smith", "status": "Interview Completed"}]'},
    {"role": "assistant", "content": "I found 2 candidates for the Data Scientist position:\n1. John Doe (ID: 67890) - Status: Applied\n2. Jane Smith (ID: 67891) - Status: Interview Completed"}
]

formatted = tokenizer.apply_chat_template(
    messages, 
    tools=custom_tools, 
    tokenize=False, 
    add_generation_prompt=False  # Don't add generation prompt for complete conversation
)

print(formatted)


<|startoftext|><|im_start|>system
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>[{"name": "get_candidate_status", "description": "Retrieves the current status of a candidate in the recruitment process", "parameters": {"type": "object", "properties": {"candidate_id": {"type": "string", "description": "Unique identifier for the candidate"}}, "required": ["candidate_id"]}}, {"name": "search_database", "description": "Search the database for specific information", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "The search query"}, "limit": {"type": "integer", "description": "Number of results to return"}}, "required": ["query"]}}]</tools>
For each function call, output the function name and arguments within the following XML format:
<tool_call>{function-name}
<arg_key>{arg_key}</arg_key>
<arg_value>{arg_value}</arg_value>
...
</tool_call>

In [11]:
# GLM-style template without think tags
glm_chat_template = (
    '{{- bos_token -}}\n'
    '{%- set system_prompt = "" -%}\n'
    '{%- set ns = namespace(system_prompt="") -%}\n'
    '{%- if messages[0]["role"] == "system" -%}\n'
    '\t{%- set ns.system_prompt = messages[0]["content"] -%}\n'
    '\t{%- set messages = messages[1:] -%}\n'
    '{%- endif -%}\n'
    '{%- if tools -%}\n'
    '\t{%- set ns.system_prompt = ns.system_prompt + ("\\n" if ns.system_prompt else "") + "# Tools\\nYou may call one or more functions to assist with the user query.\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>[" -%}\n'
    '\t{%- for tool in tools -%}\n'
    '\t\t{%- if tool is not string -%}\n'
    '\t\t\t{%- set tool = tool | tojson -%}\n'
    '\t\t{%- endif -%}\n'
    '\t\t{%- set ns.system_prompt = ns.system_prompt + tool -%}\n'
    '\t\t{%- if not loop.last -%}\n'
    '\t\t\t{%- set ns.system_prompt = ns.system_prompt + ", " -%}\n'
    '\t\t{%- endif -%}\n'
    '\t{%- endfor -%}\n'
    '\t{%- set ns.system_prompt = ns.system_prompt + "]</tools>\\nFor each function call, output the function name and arguments within the following XML format:\\n<tool_call>{function-name}\\n<arg_key>{arg_key}</arg_key>\\n<arg_value>{arg_value}</arg_value>\\n...\\n</tool_call>" -%}\n'
    '{%- endif -%}\n'
    '{%- if ns.system_prompt -%}\n'
    '\t{{- "<|im_start|>system\\n" + ns.system_prompt + "<|im_end|>\\n" -}}\n'
    '{%- endif -%}\n'
    '{%- for message in messages -%}\n'
    '\t{{- "<|im_start|>" + message["role"] + "\\n" -}}\n'
    '\t{%- set content = message["content"] -%}\n'
    '\t{%- if content is not string -%}\n'
    '\t\t{%- set content = content | tojson -%}\n'
    '\t{%- endif -%}\n'
    '\t{%- if message["role"] == "tool" -%}\n'
    '\t\t{%- set content = "<|tool_response_start|>" + content + "<|tool_response_end|>" -%}\n'
    '\t{%- endif -%}\n'
    '\t{{- content + "<|im_end|>\\n" -}}\n'
    '{%- endfor -%}\n'
    '{%- if add_generation_prompt -%}\n'
    '\t{{- "<|im_start|>assistant\\n" -}}\n'
    '{%- endif -%}'
)

tokenizer.chat_template = glm_chat_template

# Example with complete conversation flow (without think tags)
custom_tools = [
    {
        "name": "get_candidate_status", 
        "description": "Retrieves the current status of a candidate in the recruitment process", 
        "parameters": {
            "type": "object", 
            "properties": {
                "candidate_id": {"type": "string", "description": "Unique identifier for the candidate"}
            }, 
            "required": ["candidate_id"]
        }
    },
    {
        "name": "search_database",
        "description": "Search the database for specific information",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "The search query"},
                "limit": {"type": "integer", "description": "Number of results to return"}
            },
            "required": ["query"]
        }
    }
]

# Complete conversation with user, assistant, and tool messages (no think tags)
messages = [
    {"role": "user", "content": "What is the current status of candidate ID 12345?"},
    {"role": "assistant", "content": '<tool_call>get_candidate_status\n<arg_key>candidate_id</arg_key>\n<arg_value>12345</arg_value>\n</tool_call>'},
    {"role": "tool", "content": '{"candidate_id": "12345", "status": "Interview Scheduled", "position": "Clinical Research Associate", "date": "2023-11-20"}'},
    {"role": "assistant", "content": "The candidate with ID 12345 is currently in the \"Interview Scheduled\" stage for the position of Clinical Research Associate, with an interview date set for 2023-11-20."},
    {"role": "user", "content": "Can you also search for all candidates for the position of Data Scientist?"},
    {"role": "assistant", "content": '<tool_call>search_database\n<arg_key>query</arg_key>\n<arg_value>Data Scientist</arg_value>\n<arg_key>limit</arg_key>\n<arg_value>10</arg_value>\n</tool_call>'},
    {"role": "tool", "content": '[{"candidate_id": "67890", "name": "John Doe", "status": "Applied"}, {"candidate_id": "67891", "name": "Jane Smith", "status": "Interview Completed"}]'},
    {"role": "assistant", "content": "I found 2 candidates for the Data Scientist position:\n1. John Doe (ID: 67890) - Status: Applied\n2. Jane Smith (ID: 67891) - Status: Interview Completed"}
]

formatted = tokenizer.apply_chat_template(
    messages, 
    tools=custom_tools, 
    tokenize=False, 
    add_generation_prompt=False
)

print(formatted)

<|startoftext|><|im_start|>system
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>[{"name": "get_candidate_status", "description": "Retrieves the current status of a candidate in the recruitment process", "parameters": {"type": "object", "properties": {"candidate_id": {"type": "string", "description": "Unique identifier for the candidate"}}, "required": ["candidate_id"]}}, {"name": "search_database", "description": "Search the database for specific information", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "The search query"}, "limit": {"type": "integer", "description": "Number of results to return"}}, "required": ["query"]}}]</tools>
For each function call, output the function name and arguments within the following XML format:
<tool_call>{function-name}
<arg_key>{arg_key}</arg_key>
<arg_value>{arg_value}</arg_value>
...
</tool_call>

In [13]:
# Clean GLM-style template with line-by-line tool format
glm_chat_template = '''{{- bos_token -}}
{%- set system_prompt = "" -%}
{%- set ns = namespace(system_prompt="") -%}
{%- if messages[0]["role"] == "system" -%}
	{%- set ns.system_prompt = messages[0]["content"] -%}
	{%- set messages = messages[1:] -%}
{%- endif -%}
{%- if tools -%}
	{%- set ns.system_prompt = ns.system_prompt + ("\\n" if ns.system_prompt else "") + "# Tools\\nYou may call one or more functions to assist with the user query.\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>\\n" -%}
	{%- for tool in tools -%}
		{%- if tool is not string -%}
			{%- set tool = tool | tojson -%}
		{%- endif -%}
		{%- set ns.system_prompt = ns.system_prompt + tool -%}
		{%- if not loop.last -%}
			{%- set ns.system_prompt = ns.system_prompt + "\\n" -%}
		{%- endif -%}
	{%- endfor -%}
	{%- set ns.system_prompt = ns.system_prompt + "\\n</tools>\\nFor each function call, output the function name and arguments within the following XML format:\\n<tool_call>{function-name}\\n<arg_key>{arg_key}</arg_key>\\n<arg_value>{arg_value}</arg_value>\\n...\\n</tool_call>" -%}
{%- endif -%}
{%- if ns.system_prompt -%}
	{{- "<|im_start|>system\\n" + ns.system_prompt + "<|im_end|>\\n" -}}
{%- endif -%}
{%- for message in messages -%}
	{{- "<|im_start|>" + message["role"] + "\\n" -}}
	{%- set content = message["content"] -%}
	{%- if content is not string -%}
		{%- set content = content | tojson -%}
	{%- endif -%}
	{%- if message["role"] == "tool" -%}
		{%- set content = "<|tool_response_start|>" + content + "<|tool_response_end|>" -%}
	{%- endif -%}
	{{- content + "<|im_end|>\\n" -}}
{%- endfor -%}
{%- if add_generation_prompt -%}
	{{- "<|im_start|>assistant\\n" -}}
{%- endif -%}'''

tokenizer.chat_template = glm_chat_template

# Test with tools
custom_tools = [
    {
        "name": "get_candidate_status", 
        "description": "Retrieves the current status of a candidate in the recruitment process", 
        "parameters": {
            "type": "object", 
            "properties": {
                "candidate_id": {"type": "string", "description": "Unique identifier for the candidate"}
            }, 
            "required": ["candidate_id"]
        }
    }
]

messages = [
    {"role": "user", "content": "hi"},
]

formatted = tokenizer.apply_chat_template(
    messages, 
    tools=custom_tools, 
    tokenize=False, 
    add_generation_prompt=True
)
print(formatted)



# Example with complete conversation flow (without think tags)
custom_tools = [
    {
        "name": "get_candidate_status", 
        "description": "Retrieves the current status of a candidate in the recruitment process", 
        "parameters": {
            "type": "object", 
            "properties": {
                "candidate_id": {"type": "string", "description": "Unique identifier for the candidate"}
            }, 
            "required": ["candidate_id"]
        }
    },
    {
        "name": "search_database",
        "description": "Search the database for specific information",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "The search query"},
                "limit": {"type": "integer", "description": "Number of results to return"}
            },
            "required": ["query"]
        }
    }
]

# Complete conversation with user, assistant, and tool messages (no think tags)
messages = [
    {"role": "user", "content": "What is the current status of candidate ID 12345?"},
    {"role": "assistant", "content": '<tool_call>get_candidate_status\n<arg_key>candidate_id</arg_key>\n<arg_value>12345</arg_value>\n</tool_call>'},
    {"role": "tool", "content": '{"candidate_id": "12345", "status": "Interview Scheduled", "position": "Clinical Research Associate", "date": "2023-11-20"}'},
    {"role": "assistant", "content": "The candidate with ID 12345 is currently in the \"Interview Scheduled\" stage for the position of Clinical Research Associate, with an interview date set for 2023-11-20."},
    {"role": "user", "content": "Can you also search for all candidates for the position of Data Scientist?"},
    {"role": "assistant", "content": '<tool_call>search_database\n<arg_key>query</arg_key>\n<arg_value>Data Scientist</arg_value>\n<arg_key>limit</arg_key>\n<arg_value>10</arg_value>\n</tool_call>'},
    {"role": "tool", "content": '[{"candidate_id": "67890", "name": "John Doe", "status": "Applied"}, {"candidate_id": "67891", "name": "Jane Smith", "status": "Interview Completed"}]'},
    {"role": "assistant", "content": "I found 2 candidates for the Data Scientist position:\n1. John Doe (ID: 67890) - Status: Applied\n2. Jane Smith (ID: 67891) - Status: Interview Completed"}
]

formatted = tokenizer.apply_chat_template(
    messages, 
    tools=custom_tools, 
    tokenize=False, 
    add_generation_prompt=False
)

print(formatted)

<|startoftext|><|im_start|>system
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"name": "get_candidate_status", "description": "Retrieves the current status of a candidate in the recruitment process", "parameters": {"type": "object", "properties": {"candidate_id": {"type": "string", "description": "Unique identifier for the candidate"}}, "required": ["candidate_id"]}}
</tools>
For each function call, output the function name and arguments within the following XML format:
<tool_call>{function-name}
<arg_key>{arg_key}</arg_key>
<arg_value>{arg_value}</arg_value>
...
</tool_call><|im_end|>
<|im_start|>user
hi<|im_end|>
<|im_start|>assistant

<|startoftext|><|im_start|>system
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"name": "get_candidate_status", "description": "R